# Using the Emulator with Prospector

In [ ]:
import sys, os
sys.path.append(os.path.join(os.getcwd(), '..', 'src'))

import numpy as np

from prospect.fitting import fit_model
from prospect.io.write_results import write_hdf5
from prospect.plotting.utils import sample_posterior

import corner

# Load code from this repository
from utils import *
from obs import *
from models import *
from sps import *

In [ ]:
parnames = [
    "logmass",
    "logzsol",
    "logsfr_ratio_1",
    "logsfr_ratio_1",
    "logsfr_ratio_1",
    "logsfr_ratio_1",
    "logsfr_ratio_1",
    "logsfr_ratio_1",
    "dust2",
    "dust_index",
    "dust1_fraction",
    "log_fagn",
    "log_agn_tau",
    "gas_logz",
    "duste_qpah",
    "duste_umin",
    "log_duste_gamma",
]

## Fitting with Emulator

The first thing we need to do is set up the configuration settings, which get passed to the various `build_obs`, `build_sps`, `build_model`, and `fit_model` functions. The settings listed below will work for both the emulator and FSPS fits below. If you'd like to try fitting different mock objects, feel free to change `objid` (you can select any value between `1` and `10_000`).

In [ ]:
run_params = {
    # object selection
    'objid': 1,
    # emulator selection
    'emulfp': os.path.join(os.getcwd(), '..', 'data', 'parrot_v4_obsphot_256n_5l_05z30.npy'),
    # dynesty params
    'nested_maxbatch': None, # maximum number of dynamic patches
    'nested_bound': 'multi', # bounding method
    'nested_sample': 'rwalk', # sampling method
    'nested_walks': 50, # MC walks
    'nested_nlive_batch': 200, # size of live point "batches"
    'nested_nlive_init': 100, # number of initial live points
    'nested_weight_kwargs': {'pfrac': 1.0}, # weight posterior over evidence by 100%
    'nested_target_n_effective': 2000, # target effective sample size
    'nested_dlogz_init': 0.02,
    # FSPS info
    'zcontinuous': 2,
    'compute_vega_mags': False,
    'initial_disp':0.1,
    'interp_type': 'logarithmic',
    'nbins_sfh': 7,
    'df': 2,
    'agelims': [0.0,7.4772,8.0,8.5,9.0,9.5,9.8,10.0],
}

One thing to note here is that the `dynesty` settings listed above are far less stringent than those used in the paper. In the paper, we used `nested_nlive_init = 1000` and `nested_target_n_effective = 200000`. Downgrading the settings in the notebook here is just to speed up the fits by calling the emulator/FSPS fewer times - in real-world use, you'll want to use more stringent settings!

Now we can load the mock observation:

In [ ]:
obs = build_obs(**run_params)

Next we can load the SPS object.

In [ ]:
sps_emul = build_sps_emulator(**run_params)

Somewhat counterintuitively, we actually don't need this to be set to anything when using the emulator (all of the emulation stuff takes place in the model object). Hence:

In [ ]:
sps_emul

Speaking of the model object:

In [ ]:
model_emul = build_model_emulator(obs=obs, sps=sps_emul, **run_params)

One thing we need to do when fitting with the emulator is taking the emulator's uncertainties in each filter into account - to do this, we'll use the `modify_obs` function provided by this repo.

In [ ]:
obs_emul = modify_obs(obs=obs, model=model_emul)

With all that out of the way, we can now fit the mock object with the emulator. With the `dynesty` settings chosen above, this shouldn't take too long (~1 minute).

In [ ]:
output_emul = fit_model(obs_emul, model_emul, sps_emul, **run_params)

Now let's take a look at the posterior. First we'll need to resample using the sample weights.

In [ ]:
resampled_emul = sample_posterior(
    output_emul["sampling"][0]["samples"],
    weights = np.exp(output_emul["sampling"][0]["logwt"]-output_emul["sampling"][0]["logz"][-1]),
    nsample = 10_000,
)

Then we can create a corner plot:

In [ ]:
plt_emul = corner.corner(
    resampled_emul,
    labels = parnames,
    quantiles = [0.16, 0.5, 0.84],
    show_titles = True,
    truths = obs["truth"],
)

## Fitting with FSPS

In [ ]:
sps_fsps = build_sps_fsps(**run_params)

In [ ]:
model_fsps = build_model_fsps(obs=obs, sps=sps_fsps, **run_params)

Now we can actually run the fit using FSPS. This will take *much* longer (~6 hours).

*Note:* The following cell will produce many `RuntimeWarning` messages. These are caused by Prospector's `smoothspec` function and can be safely ignored in this case.

In [ ]:
output_fsps = fit_model(obs, model_fsps, sps_fsps, **run_params)

## Fitting with Stitched Emulator

As described in Section 4.3 of the paper, we've proposed a method for emulating difficult parameters where one produces multiple emulators trained over different, sligthly overlapping ranges in that parameter, and then allow the two overlapping emulators to "vote" on the photometry prediction in a differentiable way.

In this repository, we provide an example of this sort of emulator - in this case, a composite emulator composed of 24 individual emulators that collectively cover redshift for $0 < z < 24$. This is stored in the `parrot_v4_obsphot_512n_5l_24s_00z24.npy` file in the `/data` directory.

To load this, we'll slightly alter the `run_params` dictionary.

In [ ]:
run_params["emulfp"] = os.path.join(
    os.getcwd(),
    '..',
    'data',
    'parrot_v4_obsphot_512n_5l_24s_00z24.npy',
)

Then we can load the model using the `build_model_stitched` function.

In [ ]:
model_stitched = build_model_stitched(obs=obs, sps=sps_emul, **run_params)

Note that this example model allows redshift to be fit as a free variable - note that `zred` is listed as a free variable below:

In [ ]:
model_stitched

Like before, we'll need to modify the `obs` dictionary to account for the uncertainty the emulator will add to the fit. This can be done with the `modify_obs_stitched` function here:

In [ ]:
obs_stitched = modify_obs_stitched(obs=obs, model=model_stitched)

Now we can run the fit!

In [ ]:
output_stitched = fit_model(obs_stitched, model_stitched, sps_emul, **run_params)